In [1]:
from azure.identity import InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azure.ai.ml import command
from azure.ai.ml import Input, Output
from azure.ai.ml import dsl
from azureml.core import Model, Run
import os

/Users/louiedaans/anaconda3/envs/azureml/lib/python3.10/site-packages/azureml/core/__init__.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


### Connect to Azure

In [2]:
# These are your personal access credentials
credential = InteractiveBrowserCredential()

# subscription_id, resource_group, and workspace_name are strings
subscription_id = "0a94de80-6d3b-49f2-b3e9-ec5818862801"
resource_group = "buas-y2"
workspace_name = "NLP3-2025"

# Establish a connection with Azure ML
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

In [3]:
environment_name = 'ENVNLP3'
environment_version = 6
print(f'Using environment {environment_name} version {environment_version}')
compute_target_name = 'adsai-lambda-0'


env = ml_client.environments.get(environment_name, environment_version)

# @Louie - It is better to make this more flexible for us to use, let's just use 'scripts' from the current directory
# component_path = r"C:\Users\radna\OneDrive - BUas\2024-25d-fai2-adsai-RadnaPuriel234908\scripts" # Gets absolute path to local 'scripts' directory
component_path = 'scripts'

Using environment ENVNLP3 version 6


### Create and Save Components

In [4]:
# - Split data component -
datasplit_component = command(
    name="datasplit",
    display_name="Stratified Data Split",
    description="Split data into train, val, test with stratification",
    inputs={
        "data_uri": Input(type="uri_file", description="Path to input CSV"),
        "text_column_str": Input(type="string", description="Text column name"),
        "label_column_str": Input(type="string", description="Label column name"),
    },
    outputs={
        "train_uri": Output(type="uri_file", description="Path to training CSV file"),
        "val_uri": Output(type="uri_file", description="Path to validation CSV file"),
        "test_uri": Output(type="uri_file", description="Path to test CSV file"),
    },

    code=component_path,
    command="python datasplit.py "
            "--data_uri ${{inputs.data_uri}} "
            "--text_column_str ${{inputs.text_column_str}} "
            "--label_column_str ${{inputs.label_column_str}} "
            "--train_uri ${{outputs.train_uri}} "
            "--val_uri ${{outputs.val_uri}} "
            "--test_uri ${{outputs.test_uri}}",

    environment=env,
    compute=compute_target_name
)

# - Download model component - (for debugging purposes)
download_model_component = command(
    name="download_model",
    display_name="Download Model",
    description="Download model from Azure ML registry",
    inputs={
        "registered_model_name": Input(type="string", description="Name of registered model in Azure ML"),
    },
    outputs={
        "model_dir": Output(type="uri_folder", description="Path to downloaded model directory"),
    },
    code=component_path,
    command="python download_component.py "
            "--registered_model_name ${{inputs.registered_model_name}} "
            "--model_dir ${{outputs.model_dir}}",
    environment=env,
    compute=compute_target_name
)

# - Retrain data component -
train_component = command(
    name="train",
    display_name="Train Model",
    description="Train model with training and validation data from data assets",
    inputs={
        # Data
        "train_uri": Input(type="uri_file", description="Path to training CSV file"),
        "val_uri": Input(type="uri_file", description="Path to validation CSV file"),
        "text_column_str": Input(type="string", description="Name of the text column in the dataset"),
        "label_column_str": Input(type="string", description="Name of the label column in the dataset"),
        # Model
        "registered_model_name": Input(type="string", description="Name of registered model in Azure ML"),
        "epochs": Input(type="number", default=3, description="Number of training epochs"),
        "batch_size": Input(type="number", default=32, description="Batch size for training"),
        "logging_steps": Input(type="number", default=10, description="Steps between logging metrics"),
    },
    outputs={
        "model_dir": Output(type="uri_folder"),
    },
    code=component_path,
    command="python retrain.py "
            "--train_uri ${{inputs.train_uri}} "
            "--val_uri ${{inputs.val_uri}} "
            "--text_column_str ${{inputs.text_column_str}} "
            "--label_column_str ${{inputs.label_column_str}} "
            "--registered_model_name ${{inputs.registered_model_name}} "
            "--epochs ${{inputs.epochs}} "
            "--batch_size ${{inputs.batch_size}} "
            "--logging_steps ${{inputs.logging_steps}} "
            "--output_dir ${{outputs.model_dir}}",
    environment=env,
    compute=compute_target_name
)

# - Hyperparameter tuning component -
hyperparameter_tune_component = command(
    name="hyperparameter_train",
    display_name="Hyperparameter Tuning Training",
    description="Train model with given hyperparameters and log metrics",
    inputs={
        # Data
        "train_uri": Input(type="uri_file", description="Path to training CSV file"),
        "val_uri": Input(type="uri_file", description="Path to validation CSV file"),
        "text_column_str": Input(type="string", description="Name of the text column in the dataset"),
        "label_column_str": Input(type="string", description="Name of the label column in the dataset"),
        # Model
        "registered_model_name": Input(type="string", description="Name of registered model in Azure ML"),
        "epochs": Input(type="string", description="Number of training epochs"),
        "batch_size": Input(type="string", description="Batch size for training"),
        "learning_rate": Input(type="string", description="Learning rate for training"),
    },
    outputs={
        "output_dir": Output(type="uri_folder"),
    },
    code="scripts",
    command="python hyperparameter_train.py "
            "--train_uri ${{inputs.train_uri}} "
            "--val_uri ${{inputs.val_uri}} "
            "--text_column_str ${{inputs.text_column_str}} "
            "--label_column_str ${{inputs.label_column_str}} "
            "--registered_model_name ${{inputs.registered_model_name}} "
            "--epochs ${{inputs.epochs}} "
            "--batch_size ${{inputs.batch_size}} "
            "--learning_rate ${{inputs.learning_rate}} "
            "--output_dir ${{outputs.output_dir}}",
    environment=env,
    compute=compute_target_name
)

# - Evaluate model component -
evaluate_component = command(
    name="evaluate",
    display_name="Evaluate model",
    description="Evaluate model with test data and trained model",
    inputs={
        # Data
        "flex_test_uri": Input(type="uri_file", description="Path to flex test CSV file"),
        "flex_text_column_str": Input(type="string", description="Name of the text column in the flex test dataset"),
        "flex_label_column_str": Input(type="string", description="Name of the label column in the flex test dataset"),
        "static_test_uri": Input(type="uri_file", description="Path to static test CSV file"),
        "static_text_column_str": Input(type="string", description="Name of the text column in the static test dataset"),
        "static_label_column_str": Input(type="string", description="Name of the label column in the static test dataset"),
        # Eval table
        "eval_table_uri": Input(type="uri_file", description="Path to evaluation table CSV file"),
        # Model
        "model_dir": Input(type="uri_folder", description="Path to trained model directory"),
        "model_name": Input(type="string", description="Name of the model to evaluate"),
    },
    outputs={
        "outputs": Output(type="uri_folder", mode="rw_mount", description="Output directory for evaluation results"),
    },
    code=component_path,
    command="python evaluation.py "
            "--flex_test_uri ${{inputs.flex_test_uri}} "
            "--flex_text_column_str ${{inputs.flex_text_column_str}} "
            "--flex_label_column_str ${{inputs.flex_label_column_str}} "
            "--static_test_uri ${{inputs.static_test_uri}} "
            "--static_text_column_str ${{inputs.static_text_column_str}} "
            "--static_label_column_str ${{inputs.static_label_column_str}} "
            "--eval_table_uri ${{inputs.eval_table_uri}} "
            "--model_dir ${{inputs.model_dir}} "
            "--model_name ${{inputs.model_name}} "
            "--output_dir ${{outputs.outputs}}",
    environment=env,
    compute=compute_target_name
)

# - Register model component -
register_component = command(
    name="register",
    display_name="Register model",
    description="Register model if performance is above threshold",
    inputs={
        "model_dir": Input(type="uri_folder", description="Path to trained model directory"),
        "eval_results_dir": Input(type="uri_folder", description="Path to evaluation results directory"),
        "data_asset_name": Input(type="string", description="Name of the data asset used for training"),
    },
    outputs={
        "eval_table_output": Output(type="uri_file", description="Path to evaluation table CSV file"),
    },
    code=component_path,
    command="python register.py "
            "--model_dir ${{inputs.model_dir}} "
            "--eval_results_dir ${{inputs.eval_results_dir}} "
            "--data_asset_name ${{inputs.data_asset_name}} "
            "--eval_table_output ${{outputs.eval_table_output}}",
    environment=env,
    compute=compute_target_name
)

datasplit_component = ml_client.create_or_update(datasplit_component.component)
download_model_component = ml_client.create_or_update(download_model_component.component)
train_component = ml_client.create_or_update(train_component.component)
hyperparameter_tune_component = ml_client.create_or_update(hyperparameter_tune_component.component)
evaluate_component = ml_client.create_or_update(evaluate_component.component)
register_component = ml_client.create_or_update(register_component.component)

Uploading scripts (0.05 MBs): 100%|██████████| 47119/47119 [00:00<00:00, 240293.04it/s]




## Pipeline @Radna

In [7]:
#############
### BUILD ###
#############
@dsl.pipeline(
    name="Pipeline Emo Classification @Radna",
    compute="adsai-lambda-0",
)
def emotion_classifier_pipeline(
    # Data URIs
    train_data_uri: Input,
    val_data_uri: Input,
    flex_test_data_uri: Input,
    static_test_data_uri: Input,
    # Column names
    text_column_str: str,
    label_column_str: str,
    flex_text_column_str: str,
    flex_label_column_str: str,
    static_text_column_str: str,
    static_label_column_str: str,
    # Model
    registered_model_name: str,
    epochs: int,
    batch_size: int,
    logging_steps: int,
    retrained_model_name: str,
    # Evaluation and Registration
    registration_flex_threshold: float,
    registration_static_threshold: float,
):
    # Training step: train the model using the pretrained model and data
    training_step = train_component(
        train_uri=train_data_uri,
        val_uri=val_data_uri,
        text_column_str=text_column_str,
        label_column_str=label_column_str,
        epochs=epochs,
        batch_size=batch_size,
        logging_steps=logging_steps,
        registered_model_name=registered_model_name,
    )
    training_step.resources = {"instance_type": "gpu", "instance_count": 1}

    # Evaluation step: evaluate the trained model on test data
    evaluation_step = evaluate_component(
        flex_test_uri=flex_test_data_uri,
        flex_text_column_str=flex_text_column_str,
        flex_label_column_str=flex_label_column_str,
        static_test_uri=static_test_data_uri,
        static_text_column_str=static_text_column_str,
        static_label_column_str=static_label_column_str,
        model_dir=training_step.outputs.model_dir,  # <-- Output from training_step
        model_name=retrained_model_name,
    )
    evaluation_step.resources = {"instance_type": "gpu", "instance_count": 1}

    # Registration step: register the model if performance meets threshold
    register_step = register_component(
        model_dir=training_step.outputs.model_dir,
        eval_results_dir=evaluation_step.outputs.outputs,
        flex_threshold=registration_flex_threshold,
        static_threshold=registration_static_threshold,
    )

############
### PATHS ##
############
train_data_path = Input(
    type="uri_file",
    path="azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/NLP3-2025/datastores/workspaceblobstore/paths/LocalUpload/22d6fa54199d84ed8fa2c249e2d081b9/pipeline_train_data.csv", version="1"
)
val_data_path = Input(
    type="uri_file",
    path="azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/NLP3-2025/datastores/workspaceblobstore/paths/LocalUpload/42d53aaa54a20d49fbf6446107d1e9b3/pipeline_val_data.csv", version="1"
)
flex_test_data_path = Input(
    type="uri_file",
    path="azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/NLP3-2025/datastores/workspaceblobstore/paths/LocalUpload/6bed0a9544cfe3195b83d9e76d44a3a0/pipeline_test_data.xlsx", version="1"
)
static_test_data_path = Input(
    type="uri_file",
    path="azureml://subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/workspaces/NLP3-2025/datastores/workspaceblobstore/paths/UI/2025-06-14_094942_UTC/pipeline_corrected.csv", version="1"
)

###########
### RUN ###
###########
pipeline_job = emotion_classifier_pipeline(
    # Data URIs
    train_data_uri=train_data_path,
    val_data_uri=val_data_path,
    flex_test_data_uri=flex_test_data_path,
    static_test_data_uri=static_test_data_path,
    # Column names
    text_column_str="English_Sentence",
    label_column_str="Core_Emotion",
    flex_text_column_str="English_Sentence",
    flex_label_column_str="Core_Emotion",
    static_text_column_str="Translation",
    static_label_column_str='"Corrected Emotion"',
    # Model
    registered_model_name="pretrained_tranformer_model",
    epochs=10,
    batch_size=16,
    logging_steps=10,
    retrained_model_name="retrained_transformer_model-v1",
    # Evaluation and Registration
    registration_flex_threshold=0.5,
    registration_static_threshold=0.74
)
pipeline_run = ml_client.jobs.create_or_update(pipeline_job)

## Debug Pipeline @Louie

In [ ]:
################
## Components ##
################

# - Test Evaluate component -
evaluate_test_component = command(
    name="evaluate_test",
    display_name="Test Evaluate for eval_table",
    description="This is a debug component to check logic of eval_table",
    inputs={
        "eval_table_uri": Input(type="uri_file", description="Path to evaluation table CSV file"),
        "new_model_name": Input(type="string", description="Name of the model to evaluate"),
    },
    outputs={
        "eval_results": Output(type="uri_file", description="Path to evaluation table CSV file"),
    },
    code=component_path,
    command="python test_eval.py "
            "--eval_table_uri ${{inputs.eval_table_uri}} "
            "--new_model_name ${{inputs.new_model_name}} "
            "--eval_results ${{outputs.eval_results}}",
    environment=env,
    compute=compute_target_name
)

# - Test Register component -
register_test_component = command(
    name="register_test",
    display_name="Test Register for eval_table",
    description="This is a debug component to check logic of eval_table",
    inputs={
        "eval_table_uri": Input(type="uri_file", description="Path to evaluation table CSV file"),
        "data_asset_name": Input(type="string", description="Name of the data asset to register"),
    },
    outputs={
        "eval_table_output": Output(type="uri_file", description="Path to evaluation table CSV file"),
    },
    code=component_path,
    command="python test_register.py "
            "--eval_table_uri ${{inputs.eval_table_uri}} "
            "--data_asset_name ${{inputs.data_asset_name}} "
            "--eval_table_output ${{outputs.eval_table_output}}",
    environment=env,
    compute=compute_target_name
)

evaluate_test_component = ml_client.create_or_update(evaluate_test_component.component)
register_test_component = ml_client.create_or_update(register_test_component.component)

#############
### BUILD ###
#############
@dsl.pipeline(
    name="Pipeline Debug @Louie",
    compute="adsai-lambda-0",
)
def emotion_classifier_pipeline(
    eval_table_uri: Input,
    data_asset_name: str,
    new_model_name: str,
):

    # Test Evaluate step
    evaluate_test_step = evaluate_test_component(
        eval_table_uri=eval_table_uri,
        new_model_name=new_model_name,
    )

    # Test Register step
    register_test_step = register_test_component(
        eval_table_uri=evaluate_test_step.outputs.eval_results,  # <-- Output from evaluate_test_step
        data_asset_name=data_asset_name,
    )

#############
### PATHS ###
#############

# Raw data path of eval_table (latest version)
eval_table_name = "eval_table"
assets = ml_client.data.list(name=eval_table_name)
latest_version = max([int(a.version) for a in assets if a.version.isdigit()])
print(f"Using raw data version: {latest_version}")
raw_data_path = Input(
    type="uri_file",
    path=f"azureml:{eval_table_name}:{latest_version}",
)
print(raw_data_path)

#######################
### RUN AND PUBLISH ###
#######################

# Define the pipeline job
pipeline_job = emotion_classifier_pipeline(
    eval_table_uri=raw_data_path,
    data_asset_name=eval_table_name,
    new_model_name="debuging...",
)

pipeline_run = ml_client.jobs.create_or_update(pipeline_job)

Using raw data version: 4
{'type': 'uri_file', 'path': 'azureml:eval_table:4'}


## Pipeline @Louie

In [5]:
#############
### BUILD ###
#############
@dsl.pipeline(
    name="Pipeline @Louie",
    compute="adsai-lambda-0",
)
def emotion_classifier_pipeline(
    # Data URIs
    raw_data_uri: Input,
    static_test_data_uri: Input,
    # Eval table
    eval_table_uri: Input,
    data_asset_name: str,
    # Column names
    text_column_str: str,
    label_column_str: str,
    static_text_column_str: str,
    static_label_column_str: str,
    # Model
    registered_model_name: str,
    epochs: int,
    batch_size: int,
    logging_steps: int,
    retrained_model_name: str,
):
    # Data split step: split the raw data into train, val, and test sets
    data_split_step = datasplit_component(
        data_uri=raw_data_uri,
        text_column_str=text_column_str,
        label_column_str=label_column_str,
    )

    # Training step: train the model using the pretrained model and data
    training_step = train_component(
        train_uri=data_split_step.outputs.train_uri,
        val_uri=data_split_step.outputs.val_uri,
        text_column_str=text_column_str,
        label_column_str=label_column_str,
        epochs=epochs,
        batch_size=batch_size,
        logging_steps=logging_steps,
        registered_model_name=registered_model_name,
    )
    training_step.resources = {"instance_type": "gpu", "instance_count": 1}

    # Evaluation step: evaluate the trained model on test data
    evaluation_step = evaluate_component(
        flex_test_uri=data_split_step.outputs.test_uri,
        flex_text_column_str=text_column_str,
        flex_label_column_str=label_column_str,
        static_test_uri=static_test_data_uri,
        static_text_column_str=static_text_column_str,
        static_label_column_str=static_label_column_str,
        eval_table_uri=eval_table_uri,
        model_dir=training_step.outputs.model_dir,  # <-- Output from training_step
        model_name=retrained_model_name,
    )
    evaluation_step.resources = {"instance_type": "gpu", "instance_count": 1}

    # Registration step: register the model if performance meets threshold
    register_step = register_component(
        model_dir=training_step.outputs.model_dir,
        eval_results_dir=evaluation_step.outputs.outputs,
        data_asset_name=data_asset_name,
    )

#############
### PATHS ###
#############

original_model_name = "pretrained_tranformer_model"

# Raw data path (latest version)
assets = ml_client.data.list(name="new_user_corrections")
latest_version = max([int(a.version) for a in assets if a.version.isdigit()])
print(f"Using raw data version: {latest_version}")
raw_data_path = Input(
    type="uri_file",
    path=f"azureml:new_user_corrections:{latest_version}",
)

# Static test data path (latest version)
assets = ml_client.data.list(name="static_test")
latest_version = max([int(a.version) for a in assets if a.version.isdigit()])
print(f"Using static test data version: {latest_version}")
static_test_data_path = Input(
    type="uri_file",
    path=f"azureml:static_test:{latest_version}",
)

# Raw data path of eval_table (latest version)
eval_table_name = "eval_table"
assets = ml_client.data.list(name=eval_table_name)
latest_version = max([int(a.version) for a in assets if a.version.isdigit()])
print(f"Using raw data version: {latest_version}")
raw_eval_table_path = Input(
    type="uri_file",
    path=f"azureml:{eval_table_name}:{latest_version}",
)
print(raw_data_path)

###########
### RUN ###
###########
pipeline_job = emotion_classifier_pipeline(
    # Data URIs
    raw_data_uri=raw_data_path,
    static_test_data_uri=static_test_data_path,
    # Eval table
    eval_table_uri=raw_eval_table_path,
    data_asset_name=eval_table_name,
    # Column names
    text_column_str="Translation",
    label_column_str="Emotion",
    static_text_column_str="Translation",
    static_label_column_str='"Corrected Emotion"',
    # Model
    registered_model_name=original_model_name,
    epochs=20,
    batch_size=8,
    logging_steps=5,
    retrained_model_name="final_test_v1",
)

pipeline_run = ml_client.jobs.create_or_update(pipeline_job)

Using raw data version: 2
Using static test data version: 1


Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Using raw data version: 9
{'type': 'uri_file', 'path': 'azureml:new_user_corrections:2'}


## Pipeline Hyp @Louie

In [11]:
#############
### BUILD ###
#############
@dsl.pipeline(
    name="Pipeline Hyp Emo Classification @Louie",
    compute="adsai-lambda-0",
)
def emotion_classifier_pipeline(
    # Data URIs
    raw_data_uri: Input,
    static_test_data_uri: Input,
    # Column names
    text_column_str: str,
    label_column_str: str,
    static_text_column_str: str,
    static_label_column_str: str,
    # Model
    registered_model_name: str,
    epochs: str,           # <-- Now string for comma-separated values
    batch_size: str,       # <-- Now string for comma-separated values
    learning_rate: str,    # <-- New: comma-separated learning rates
    retrained_model_name: str,
    # Evaluation and Registration
    registration_flex_threshold: float,
    registration_static_threshold: float,
):
    # Data split step: split the raw data into train, val, and test sets
    data_split_step = datasplit_component(
        data_uri=raw_data_uri,
        text_column_str=text_column_str,
        label_column_str=label_column_str,
    )

    # Hyperparameter tuning training step
    hyper_training_step = hyperparameter_tune_component(
        train_uri=data_split_step.outputs.train_uri,
        val_uri=data_split_step.outputs.val_uri,
        text_column_str=text_column_str,
        label_column_str=label_column_str,
        epochs=epochs,
        batch_size=batch_size,
        learning_rate=learning_rate,
        registered_model_name=registered_model_name,
    )
    hyper_training_step.resources = {"instance_type": "gpu", "instance_count": 1}

    # Evaluation step: evaluate the best model on test data
    evaluation_step = evaluate_component(
        flex_test_uri=data_split_step.outputs.test_uri,
        flex_text_column_str=text_column_str,
        flex_label_column_str=label_column_str,
        static_test_uri=static_test_data_uri,
        static_text_column_str=static_text_column_str,
        static_label_column_str=static_label_column_str,
        model_dir=hyper_training_step.outputs.output_dir,  # <-- Output from hyperparameter_tune_component
        model_name=retrained_model_name,
    )
    evaluation_step.resources = {"instance_type": "gpu", "instance_count": 1}

    # Registration step: register the model if performance meets threshold
    register_step = register_component(
        model_dir=hyper_training_step.outputs.output_dir,
        eval_results_dir=evaluation_step.outputs.outputs,
        flex_threshold=registration_flex_threshold,
        static_threshold=registration_static_threshold,
    )

#############
### PATHS ###
#############

original_model_name = "pretrained_tranformer_model"

# Raw data path (latest version)
assets = ml_client.data.list(name="new_user_corrections")
latest_version = max([int(a.version) for a in assets if a.version.isdigit()])
print(f"Using raw data version: {latest_version}")
raw_data_path = Input(
    type="uri_file",
    path=f"azureml:new_user_corrections:{latest_version}",
)

# Static test data path (latest version)
assets = ml_client.data.list(name="static_test")
latest_version = max([int(a.version) for a in assets if a.version.isdigit()])
print(f"Using static test data version: {latest_version}")
static_test_data_path = Input(
    type="uri_file",
    path=f"azureml:static_test:{latest_version}",
)

###########
### RUN ###
###########
pipeline_job = emotion_classifier_pipeline(
    # Data URIs
    raw_data_uri=raw_data_path,
    static_test_data_uri=static_test_data_path,
    # Column names
    text_column_str="Translation",
    label_column_str="Emotion",
    static_text_column_str="Translation",
    static_label_column_str='"Corrected Emotion"',
    # Model
    registered_model_name=original_model_name,
    epochs="30",                # Example
    batch_size="4,8",             # Example
    learning_rate="1e-5,1e-8",     # Example
    retrained_model_name="retrained_transformer_model-v1",
    # Evaluation and Registration
    registration_flex_threshold=0.5,
    registration_static_threshold=0.74
)

pipeline_run = ml_client.jobs.create_or_update(pipeline_job)

Using raw data version: 1
Using static test data version: 1
